In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import IPython
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

# InteractiveShell.ast_node_interactivity = "all"
# CSS = """
# .output {
#     flex-direction: row;
# }
# """
# HTML('<style>{}</style>'.format(CSS))
# pd.set_option('display.max_columns', None)

---

## Mapping categories to DF1 and removing outliers below

In [3]:
df1 = pd.read_csv('./data/0826export_column_filter_1.csv')
df1.columns = df1.columns.str.lower()

df2 = pd.read_excel('./documentation/store_master.xlsx')
df2.columns = df2.columns.str.lower()

categories_to_map = ['class', 
                     'store_city', 
                     'store_state', 
                     'close_date', 
                     'gross_feet'] # must be df2's LOWERED column names

def map_my_dataframe(df1, df2, a_list):
    """Returns DataFrame(df1)
    maps str items in a_list, from df2 onto df1.
    the joint variable for mapping is the store number.
    
    !! df1 and df2 column names must be lower-case !!
    """
    joint_var = 'store'
    for category in a_list:
        df_2_dict = dict(zip(df2[joint_var], df2[category]))
        df1[category] = df1[joint_var].map(df_2_dict)
    return df1
    
df = map_my_dataframe(df1, df2, categories_to_map)
df = df[df.store != 2084]

closed = df[df.close_date.notna()]

---

### Remove outlier store(2084), make DF for closed stores.

---

## Compare all variables by top/bot `COMP_PCT` by `STORE`:

Grouping DataFrames's by `'store'`, `'mean()'` of:
 - `df1`: full original data frame
 - `top_avg`: top by `comp_pct`
 - `bot_avg`: bottom by `comp_pct`
 - `closed_avg`: by closed store

In [157]:
all_avg_by_store = df.groupby(['store']).mean().mean().round(4)
top_avg_by_store = df.groupby(['store']).mean().nlargest(20, 'comp_pct').mean().round(4)
bot_avg_by_store = df.groupby(['store']).mean().nsmallest(20, 'comp_pct').mean().round(4)
closed_avg_by_store = closed.groupby(['store']).mean().mean().round(4)

# Concatenating
for_compare_by_store = pd.concat([
#     all_avg_by_store.rename('all_means').to_frame(),
    top_avg_by_store.rename('top_store_avgs').to_frame(),
    bot_avg_by_store.rename('bot_store_avgs').to_frame(),
    closed_avg_by_store.rename('closed_store_avgs').to_frame()
], axis=1).iloc[3:, :]

# for_compare_by_store

store_top_bot_diff = abs(abs(top_avg_by_store - bot_avg_by_store) / ((top_avg_by_store + bot_avg_by_store) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[2:, :]
store_top_bot_diff

store_top_closed_diff = abs(abs(top_avg_by_store - closed_avg_by_store) / ((top_avg_by_store + closed_avg_by_store) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[2:, :]
store_top_closed_diff

,abs_%_diff
avg_ft_tenure_days,107.119953
special_order_amt,35.028947
avg_pt_tenure_days,32.765124
trans_cnt_returns,28.547131
accessory_value,27.532917
avg_mgr_tenure_days,25.391116
sales_value,21.943832
strak_sales_amt,21.930740
mark_down_amt_ty,21.545275


,abs_%_diff
mark_down_amt_ty,59.988142
avg_mgr_tenure_days,49.948303
special_order_amt,40.877791
avg_ft_tenure_days,40.092304
accessory_value,39.324966
shoes_units,36.247024
trans_cnt_returns,33.627098
strak_traffic,32.747220
sales_value,30.338543


---
## Filter `CLASS`, Compare by top/bot `COMP_PCT` by [`STORE`, `FISCAL_YEAR`] -->  `FISCAL_MONTH`:

### Commented out classes do not have enough stores to do top and bot 10 store difference checks

In [128]:
classes = df['class'].unique()

mall_top = df[df['class'] == classes[0]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('mall_top').to_frame()
mall_bot = df[df['class'] == classes[0]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('mall_bot').to_frame()

# open_air_top = df[df['class'] == classes[1]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('open_air_top').to_frame()
# open_air_bot = df[df['class'] == classes[1]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('open_air_bot').to_frame()

# downtown_top = df[df['class'] == classes[2]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('downtown_top').to_frame()
# downtown_bot = df[df['class'] == classes[2]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('downtown_bot').to_frame()

# street_top = df[df['class'] == classes[3]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('street_top').to_frame()
# street_bot = df[df['class'] == classes[3]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('street_bot').to_frame()

airport_top = df[df['class'] == classes[4]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('airport_top').to_frame()
airport_bot = df[df['class'] == classes[4]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('airport_bot').to_frame()

# lifestyle_top = df1[df1['class'] == classes[5]].groupby(['store']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('lifestyle_top').to_frame()
# lifestyle_bot = df1[df1['class'] == classes[5]].groupby(['store']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('lifestyle_bot').to_frame()

outlet_top = df[df['class'] == classes[6]].groupby(['store','fiscal_year','fiscal_month']).mean().nlargest(10, 'comp_pct').iloc[:, 2:].mean().rename('outlet_top').to_frame()
outlet_bot = df[df['class'] == classes[6]].groupby(['store','fiscal_year','fiscal_month']).mean().nsmallest(10, 'comp_pct').iloc[:, 2:].mean().rename('outlet_bot').to_frame()

# for_compare_by_class = pd.concat([mall_top, mall_bot, airport_top, airport_bot, outlet_top, outlet_bot], axis=1)
# for_compare_by_class

In [156]:
outlet_top = df[df['class'] == classes[6]].groupby(['store','fiscal_year','fiscal_month']).mean().iloc[:, 1:].nlargest(500, 'comp_pct').mean()
outlet_bot = df[df['class'] == classes[6]].groupby(['store','fiscal_year','fiscal_month']).mean().iloc[:, 1:].nsmallest(500, 'comp_pct').mean()


outlet_class_diff = abs(abs(outlet_top - outlet_bot) / ((outlet_top + outlet_bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[2:, :]
outlet_class_diff

,abs_%_diff
avg_ft_tenure_days,44.233755
mark_down_amt_ty,27.312015
multi_value,26.103601
special_order_amt,24.926410
shoes_units,22.870279
shoes_value,21.075789
sales_value,19.880310
strak_sales_amt,19.824894
trans_cnt_returns,18.441151


---

## MALL STORES:
 - #### Absolute percentage(%) differences between averages of top and bot mall stores' variables:

In [10]:
top = mall_top.iloc[1:,0]
bot = mall_bot.iloc[1:,0]
diff_mall = abs(abs(bot - top) / ((bot + top) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]
diff_mall

,abs_%_diff
avg_am_tenure_days,55.178889
avg_pt_tenure_days,46.603252
avg_ft_tenure_days,40.538596
avg_mgr_tenure_days,36.306432
special_order_amt,35.176158
multi_value,33.870609
shoes_value,27.212221
trans_cnt_returns,26.260592
shoes_units,25.909604
mark_down_amt_ty,25.493565


---

## AIRPORT STORES:
 - #### Absolute percentage(%) differences between averages of top and bot airport stores' variables:

In [11]:
top = airport_top.iloc[1:,0]
bot = airport_bot.iloc[1:,0]
diff_airport = abs(abs(bot-top) / ((bot + top) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]
diff_airport

,abs_%_diff
avg_ft_tenure_days,118.674498
avg_am_tenure_days,64.814526
avg_mgr_tenure_days,62.508963
trans_cnt_returns,56.637767
tot_avg_tenure_days,46.210015
mark_down_amt_ty,33.349293
multi_value,32.473314
accessory_value,31.384410
strak_traffic,30.956370
shoes_units,30.580276


---

## OUTLET STORES: 
 - #### Absolute percentage(%) differences between averages of top and bot outlet stores' variables:

In [12]:
top = outlet_top.iloc[1:,0]
bot = outlet_bot.iloc[1:,0]

# outlet store differences
diff_outlet = abs(abs(top-bot) / ((top + bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(10, 'abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]
diff_outlet

,abs_%_diff
avg_mgr_tenure_days,86.448324
avg_pt_tenure_days,62.505554
tot_avg_tenure_days,61.796806
trans_cnt_returns,24.976449
shoes_value,20.087996
shoes_units,19.149166
sales_value,16.211427
strak_sales_amt,16.188806
strak_shopper_to_assoc_ratio,15.427341


---

## Compare top/bot `COMP_PCT` by [`STORE`, `FISCAL_YEAR`] --> `FISCAL_MONTH`: 
 - #### Absolute percentage(%) differences between averages of top and bot fiscal months':

In [111]:
grouped_store_year_month = df.groupby(['store', 'fiscal_year', 'fiscal_month']).mean().iloc[:, 1:]

top_store_month = grouped_store_year_month.nlargest(1000, 'comp_pct').mean()
bot_store_month = grouped_store_year_month.nsmallest(1000, 'comp_pct').mean()

diff_store_month = abs(abs(top_store_month - bot_store_month) / ((top_store_month + bot_store_month) * 0.5) * 100).nlargest(11).iloc[2:]

diff_store_month

special_order_amt         34.738222
accessory_value           32.049302
multi_value               31.437389
mark_down_amt_ty          28.710249
sales_value               27.646837
strak_sales_amt           27.642270
shoes_value               25.102516
shoes_units               24.308150
sales_amt_per_pay_hour    22.247581
dtype: float64

In [109]:
by_store_year = df.groupby([
    'store', 
    'fiscal_year', 
    'fiscal_month',
    'fiscal_week'
]).mean()#.iloc[:, 2:]

# store_top['comp_pct_year'] = 

# store_top#.nlargest(5, 'comp_pct')

store_top = by_store_year.nlargest(100, 'comp_pct').mean()
store_bot = by_store_year.nsmallest(100, 'comp_pct').mean()

diff_store = abs(abs(store_top - store_bot) / ((store_top + store_bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[2:, :]
diff_store
# by_store_year

,abs_%_diff
accessory_value,107.086036
multi_value,96.906436
mark_down_amt_ty,93.225881
sales_value,92.788071
strak_sales_amt,92.465483
shoes_value,85.516619
shoes_units,82.479707
special_order_amt,82.314735
sales_amt_per_pay_hour,81.997074


---

## Compare top/bot `COMP_PCT` by [`STORE`, `FISCAL_YEAR`, `FISCAL_MONTH`] --> `FISCAL_WEEK`: 
 - #### Absolute percentage(%) differences between averages of top and bot fiscal weeks' variables:

In [110]:
wk_top = fiscal_wk_top.mean()[1:]
wk_bot = fiscal_wk_bot.mean()[1:]

diff_week = abs(abs(wk_top - wk_bot) / ((wk_top + wk_bot) * 0.5) * 100).to_frame('abs_%_diff').nlargest(11, 'abs_%_diff').iloc[1:, :]
diff_week

,abs_%_diff
comp_pct,267.763589
accessory_value,138.462062
mark_down_amt_ty,121.847862
sales_value,119.909024
strak_sales_amt,119.490504
multi_value,117.367459
special_order_amt,117.125208
shoes_units,111.017662
shoes_value,109.962029
sales_amt_per_pay_hour,106.314500


special_order_amt         34.738222
accessory_value           32.049302
multi_value               31.437389
mark_down_amt_ty          28.710249
sales_value               27.646837
strak_sales_amt           27.642270
shoes_value               25.102516
shoes_units               24.308150
sales_amt_per_pay_hour    22.247581
dtype: float64